In [ ]:
#本代码用vgg16在imagenet预训练的权重提取图像特征并写入txt文件
#图像只接受三通道，png可能报错，灰度图也不行，png转jpg代码

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os, glob
import h5py
import time
from tqdm import tqdm

In [3]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        VGG = models.vgg16(pretrained=True)#使用与训练权重
        self.feature = VGG.features
    def forward(self, x):
        output = self.feature(x)#输出维度为（512*7*7）
        m = nn.MaxPool2d(7, stride=1)
        output = m(output)#对输出的维度进行平均池化->（512*1*1）
        output = output.view(output.size(0), -1)#512*1
        return output

In [4]:
model = Encoder()#定义模型
def extractor(img_path, net, use_gpu, file_name):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()]
    )#将图像转化为张量的形式，同时扩充224*224
    img = Image.open(img_path)#读取图像
    img = transform(img)
    x = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)
    if use_gpu:
        x = x.cuda()
        net = net.cuda()
    y = net(x).cpu()
    y = torch.squeeze(y)
    ys = y.data.numpy()
    file_names = file_name
    return file_names, ys


d:\ana_conda\envs\PMT\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\ana_conda\envs\PMT\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
#这块可以跑通输出写有特征的txt文件,使用 另存为 功能转为csv文件即可
if __name__ == '__main__':
    #data_dir = 'E:/3_食品塑料包装\食品塑料包装\picture1/1_3684d_原始.png'#图像路径
    data_dir = 'E:/480360data/480360_71'#图像路径，该文件夹下不能直接是图片，而要一个文件夹，里面是才是图片
    files_list = []
    names = []
    features = []
    x = os.walk(data_dir)
    #print(x)
    for path, d, filelist in x:
        print(filelist)
        for filename in filelist:
            #print(filename)
            file_glob = os.path.join(path, filename)
            files_list.extend(glob.glob(file_glob))
    use_gpu = torch.cuda.is_available()
    pbar = tqdm(files_list)
    for x_path in files_list and pbar:
        file_name = x_path.split('/')[-1]
        #print(file_name)
        file_names, ys = extractor(x_path, model, use_gpu, file_name)
        features.append(ys.tolist())
        names.append(file_names)
        pbar.set_description("Extracting features")
        # 设置进度条右边显示的信息
        pbar.set_postfix(Img_name=file_names)
        time.sleep(0.1)
        # 文件路径
    file_path = 'features_480360_71.txt'###修改txt名称

    # 使用'w'模式打开文件，如果文件不存在则创建，如果存在则覆盖原有内容
    # 使用'a'模式打开文件，如果文件不存在则创建，如果存在则接着原有内容补充
    with open(file_path, 'a', encoding='utf-8') as file:
        for i in range(len(features)):
            #file.write("\n")
            a=str(features[i]).replace("[","")
            b=a.replace("]","")
            file.write(b)
            file.write("\n")

    
    print(f'文本已写入到文件：{file_path}')


[]
['10_3693d_差分.jpg', '111111111111111_3683d_差分.jpg', '11_3694d_差分.jpg', '12-1_3737d_差分.jpg', '12_3695d_差分.jpg', '13_3696d_差分.jpg', '14_3697d_差分.jpg', '15-1_3740d_差分.jpg', '15_3698d_差分.jpg', '16-1_3700d_差分.jpg', '16_3699d_差分.jpg', '18_3701d_差分.jpg', '19_3702d_差分.jpg', '1_3684d_差分.jpg', '2-1_3736d_差分.jpg', '20_3703d_差分.jpg', '21_3704d_差分.jpg', '22-1_3739d_差分.jpg', '22_3705d_差分.jpg', '23_3706d_差分.jpg', '24_3707d_差分.jpg', '25_3708d_差分.jpg', '26_3709d_差分.jpg', '27_3710d_差分.jpg', '28_3711d_差分.jpg', '29_3712d_差分.jpg', '2_3685d_差分.jpg', '3-1_3738d_差分.jpg', '30_3713d_差分.jpg', '31_3714d_差分.jpg', '32_3715d_差分.jpg', '33_3716d_差分.jpg', '34_3717d_差分.jpg', '35_3718d_差分.jpg', '36_3719d_差分.jpg', '37_3720d_差分.jpg', '38_3721d_差分.jpg', '39_3722d_差分.jpg', '3_3686d_差分.jpg', '40-1_3743d_差分.jpg', '40_3723d_差分.jpg', '41_3724d_差分.jpg', '42_3725d_差分.jpg', '43_3726d_差分.jpg', '44_3727d_差分.jpg', '45_3728d_差分.jpg', '46-1_3745d_差分.jpg', '46_3729d_差分.jpg', '47_3730d_差分.jpg', '48-1_3746d_差分.jpg', '48_3731d_差分.jpg', '

Extracting features: 100%|██████████| 71/71 [00:10<00:00,  6.90it/s, Img_name=480360_71\480360 _71\铝片_3744d_差分.jpg] ]          

文本已写入到文件：features_480360_71.txt
